In [1]:
import itertools
import importlib
import yaml
import sys
import pickle
import multiprocessing as mp
import heapq
from datetime import datetime
import pandas as pd
from tqdm import tqdm
import numpy as np
import math
from fastdtw import fastdtw
import sys

sys.path.insert(0, './code')\
# Local Imports
from TDA import TDA_Parameters, ts_to_tda
from CrossCorrelation import cross_cor
from Metrics import recall_at_k, heap_to_ordered_list, get_value_position
from Preprocess import preprocess
from CastCol import cast_columns

In [2]:

config_file = "./code/config/dns.yaml"
with open(config_file, 'r') as file:
    config = yaml.safe_load(file)
# ==============================================================================
# END Config
# ==============================================================================

window = pd.Timedelta(config['window'])
num_cpus = config['num_cpus']

module = importlib.import_module('ScopeFilters')
for scope in config['scope_config']:
    scope[2] = getattr(module, scope[2])

tda_config = TDA_Parameters(config['dim'],
                            config['tda_window'],
                            config['skip'],
                            config['k'],
                            float(config['thresh']))


src, dst = preprocess("./code/" + config['pcappath'],
                      "./code/" + config['logpath'],
                      config['scope_config'],
                      config['server_logs'],
                      config['infra_ip'],
                      window,
                      config['evil_domain'],
                      config['bad_features'],
                      debug=config['DEBUG'])

./code/DFutil.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'count'] = 1
./code/DFutil.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'count'] = 1
./code/DFutil.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'count'] = 1
.

['102.0.0.72', '102.0.0.83', '102.0.0.80', '102.0.0.43', '102.0.0.17', '102.0.0.46', '102.0.0.19', '102.0.0.51', '102.0.0.28', '102.0.0.10', '102.0.0.77', '102.0.0.76', '102.0.0.44', '102.0.0.74', '102.0.0.95', '102.0.0.24', '102.0.0.37', '102.0.0.49', '102.0.0.109', '102.0.0.94', '102.0.0.73', '102.0.0.96', '102.0.0.27', '102.0.0.92', '102.0.0.85', '102.0.0.36', '102.0.0.47', '102.0.0.71', '102.0.0.69', '102.0.0.34', '102.0.0.30', '102.0.0.102', '102.0.0.14', '102.0.0.21', '102.0.0.81', '102.0.0.12', '102.0.0.88', '102.0.0.91', '102.0.0.39', '102.0.0.90', '102.0.0.103', '102.0.0.38', '102.0.0.35', '102.0.0.105', '102.0.0.65', '102.0.0.100', '102.0.0.99', '102.0.0.52', '102.0.0.45', '102.0.0.11', '102.0.0.20', '102.0.0.57', '102.0.0.82', '102.0.0.78', '102.0.0.61', '102.0.0.16', '102.0.0.106', '102.0.0.93', '102.0.0.55', '102.0.0.62', '102.0.0.33', '102.0.0.25', '102.0.0.13', '102.0.0.86', '102.0.0.50', '102.0.0.40', '102.0.0.26', '102.0.0.58', '102.0.0.67', '102.0.0.89', '102.0.0.75',

./code/Solo.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['index_col'] = new_df.index
./code/Solo.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['diff'] = new_df['index_col'].diff()
./code/Solo.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-c

In [5]:
flows_ts_ip_total = src
client_chat_logs = dst
bad_features = ['ip', 'udp.dstport', 'frame', 'tcp.dstport', 'tcp.ack', 'tcp.time_relative', 'tcp.reassembled.length']

for ip in flows_ts_ip_total:
    cast_columns(flows_ts_ip_total[ip])
    df = flows_ts_ip_total[ip]
    cols_to_remove = []
    for pattern in bad_features:
        cols_to_remove += [c for c in df.columns if pattern in c.lower()]
    df.drop(columns=cols_to_remove, inplace=True)

for user in client_chat_logs:
    cast_columns(client_chat_logs[user])

In [8]:
flows_ts_ip_total['102.0.0.73'].columns


Index(['count', 'count_resolver', 'tcp.len', 'tcp.time_delta', 'tcp.len_ISP',
       'tcp.time_delta_ISP', 'count_ISP', 'count_cache', 'count_cache_root',
       'count_cache_SLD', 'count_cache_TLD', 'tcp.len_Access_service',
       'tcp.time_delta_Access_service', 'count_Access_service',
       'tcp.len_Service', 'tcp.time_delta_Service', 'count_Service'],
      dtype='object')

In [ ]:
# def ip_to_group(ip):
#     if isinstance(ip, float) or ip.split(".")[0] != '102':
#         return -1
#     return math.floor((int(ip.split(".")[-1])-2) / 5)


# def get_real_label(dic):
#     data = dic.keys()
#     result = np.array([ip_to_group(xi) for xi in data])
#     return result


# answers = get_real_label(flows_ts_ip_total)


def my_dtw(ts1, ts2):
    distance, path = fastdtw(ts1, ts2)
    return distance


def my_pl_ts(ts1, ts2, ip1, ip2):
    return my_dtw(ts1, ts2)


def my_dist(ts1, ts2, ip1="", ip2=""):
    return my_pl_ts(ts1, ts2, ip1, ip2)


def ip_to_user(ip, group_size=5, starting=5):
    isp = int(int(ip.split(".")[-2]))
    node_number = int(ip.split(".")[-1]) - starting - isp
    user = node_number % group_size
    group = math.floor(node_number / group_size)
    return '/tordata/config/group_' + str(group) + "_user_" + str(user)


def compare_ts(ts1, ts2, debug=False):
    # dtw_classic, path_classic = dtw(ts1, ts2, dist='square',
    #                             method='classic', return_path=True)
    # return dtw_classic
    # print(ts1)
    # print(ts2)
    # dist, lag = cross_cor(pd.Series(ts1), pd.Series(ts2))
    dist, lag = cross_cor(ts1, ts2, debug=debug)
    # assert dist >= -1 and dist <= 1
    dist = dist * -1  # flip for use as distance metric
    # assert dist >= -1 and dist <= 1
    return dist, lag


def normalize_ts(ts):
    ts = (ts-ts.min())/(ts.max()-ts.min())
    return ts.fillna(0)


def compare_ts_reshape(ts1, ts2, debug=False):
    # buffer_room = 120  # in seconds
    range = min(ts2.index.values), max(ts2.index.values)
    ts1 = ts1.loc[(ts1.index >= range[0]) & (ts1.index <= range[1])]
    # ts1 = ts1[(ts1['frame.time'] >= int(range[0])) &
    #           (ts1['frame.time'] <= int(range[1]))]
    # print(ts1)
    # ts1 = ts1.loc[:, 'tda_pl']
    ts1 = ts1.values[:, 0]

    ts1_norm = np.array(ts1)
    ts2_norm = np.array(ts2)

    # delay = 0

    # ts1_norm.index = ts1_norm.index + pd.DateOffset(seconds=delay)

    # lock to same range with buffer room
    # on each side to account for network (or PPT) delay

    # detect if no overlap
    if len(ts1_norm) < 2 or len(ts2_norm) < 2:
        return float("inf"), 0

    # Normalize peaks?
    # ts1_norm = normalize_ts(ts1_norm)
    # ts2_norm = normalize_ts(ts2_norm)

    # plot_ts(ts1_norm, ts2_norm)
    # exit(1)

    # else:
    #     ts1_norm = ts1_norm.tolist()
    #     ts2_norm = ts2_norm.tolist()

    score, lag = compare_ts(ts1_norm, ts2_norm, debug=debug)

    return score, lag


def evaluate(src_raw, dst_raw, src_features, dst_feaures, display=False, params=TDA_Parameters(0, 3, 1, 1, 1)):
    src = {}
    dst = {}
    for ip in src_raw:
        try:
            data = src_raw[ip][src_features].copy(deep=True)
        except Exception:
            data = pd.DataFrame(0, index=src_raw[ip].index, columns=src_features)
        if config['tda']:
            src[ip] = ts_to_tda(data)
        else:
            src[ip] = data
    for user in dst_raw:
        try:
            data = dst_raw[user][dst_feaures].copy(deep=True)
        except Exception:
            data = pd.DataFrame(0, index=dst_raw[user].index, columns=dst_feaures)
        if config['tda']:
            dst[user] = ts_to_tda(data)
        else:
            dst[user] = data

    correct = 0.0
    rank_list = []
    score_list = []
    recall_2 = 0
    recall_4 = 0
    recall_8 = 0
    rank = 0
    for user in dst:
        best_score = 0
        best_user = 0
        heap = []
        counter = 0
        r2 = False
        r4 = False
        r8 = False
        for ip in src:
            counter += 1
            score, _ = compare_ts_reshape(src[ip], dst[user])
            if not math.isnan(score) and not math.isinf(score):
                heapq.heappush(heap, (score, counter, ip_to_user(ip)))
            if score < best_score:
                best_score = score
                best_user = ip_to_user(ip)
        if user == best_user:
            correct += 1
        # print(user)
        if recall_at_k(heap.copy(), 2, user):
            recall_2 += 1
            r2 = True
        if recall_at_k(heap.copy(), 4, user):
            recall_4 += 1
            r4 = True
        if recall_at_k(heap.copy(), 8, user):
            recall_8 += 1
            r8 = True
        if (r2 and (not r4 or not r8)) or (r4 and not r8):
            print("r2: " + str(r2))
            print("r4: " + str(r4))
            print("r8: " + str(r8))
            raise Exception("Bad recall")
        rank += get_value_position(heap, user)
        rank_list += [(get_value_position(heap, user), user)]
        score_list += [(heap_to_ordered_list(heap), user)]
    accuracy = correct / len(src)
    recall_2 = recall_2 / len(src)
    recall_4 = recall_4 / len(src)
    recall_8 = recall_8 / len(src)
    rank = rank / len(src)
    return accuracy, recall_2, recall_4, recall_8, rank, rank_list, score_list


def findsubsets(s, n):
    return list(itertools.combinations(s, n))


def evaluate_subset(src_df, dst_df, src_features, dst_feaures, tda_config=None):
    score = evaluate(src_df, dst_df, list(src_features), list(dst_feaures), params=tda_config)
    return score, src_features


def get_features(df):
    features = []
    for src in df:
        features += df[src].columns.tolist()
    return list(set(features))


def iterate_features(src_df, dst_df, n, dst_features, tda_config, filename):
    features = get_features(src_df)
    subsets = findsubsets(features, n)
    results = []
    print("Using " + str(num_cpus) + " cpus for " + str(len(subsets)) + " subsets")
    with mp.Pool(processes=num_cpus) as pool:
        results = []
        for subset in subsets:
            results.append(pool.apply_async(evaluate_subset, args=(src_df, dst_df, subset, dst_features, tda_config)))
        with open(filename, 'a+') as f:
            for result in tqdm(results, total=len(subsets)):
                score, subset = result.get()
                out = str(score) + "\t" + str(subset) + "\n"
                f.write(out)


src_df = flows_ts_ip_total
dst_df = client_chat_logs

# dst_features = ['count']
# src_features = ['count']
# n = 1
# data = evaluate_subset(src_df, dst_df, src_features, dst_features)[-2][-1]
# with open(output_file, 'w') as f:
#     for i in data:
#         out = str(i[-1]) + ", " + str(i[0]) + "\n"
#         f.write(out)

for output_size in range(1, len(dst_df)+1):
    for n in range(1, 3):
        for features in findsubsets(get_features(dst_df), output_size):
            print("Evaluating " + str(n) + " features from " + str(output_size) + " output features")
            best_features = iterate_features(src_df, dst_df, n, features, tda_config,
                                             config['experiment_name'] + str(n) +
                                             "_outputFeatures_" + str(features) +
                                             "_" + str(datetime.now()) +
                                             ".output")
import itertools
import importlib
import yaml
import sys
import pickle
import multiprocessing as mp
import heapq
from datetime import datetime
import pandas as pd
from tqdm import tqdm
import numpy as np
import math
from fastdtw import fastdtw

# Local Imports
from TDA import TDA_Parameters, ts_to_tda
from CrossCorrelation import cross_cor
from Metrics import recall_at_k, heap_to_ordered_list, get_value_position
from Preprocess import preprocess
from CastCol import cast_columns


# ==============================================================================
# Config
# ==============================================================================
if len(sys.argv) < 2:
    print("Usage: python {} <config.yaml>".format(sys.argv[0]))
    sys.exit(1)

config_file = sys.argv[1]
with open(config_file, 'r') as file:
    config = yaml.safe_load(file)
# ==============================================================================
# END Config
# ==============================================================================

window = pd.Timedelta(config['window'])
num_cpus = config['num_cpus']

module = importlib.import_module('ScopeFilters')
for scope in config['scope_config']:
    scope[2] = getattr(module, scope[2])

tda_config = TDA_Parameters(config['dim'],
                            config['tda_window'],
                            config['skip'],
                            config['k'],
                            float(config['thresh']))


src, dst = preprocess(config['pcappath'],
                      config['logpath'],
                      config['scope_config'],
                      config['server_logs'],
                      config['infra_ip'],
                      window,
                      config['evil_domain'],
                      config['bad_features'],
                      debug=config['DEBUG'])

p_filename = config['experiment_name'] + "_ts.pkl"
with open(p_filename, 'wb') as file:
    pickle.dump(src, file)
    pickle.dump(dst, file)

with open(p_filename, 'rb') as file:
    flows_ts_ip_total = pickle.load(file)
    client_chat_logs = pickle.load(file)

for ip in flows_ts_ip_total:
    cast_columns(flows_ts_ip_total[ip])

for user in client_chat_logs:
    cast_columns(client_chat_logs[user])


# def ip_to_group(ip):
#     if isinstance(ip, float) or ip.split(".")[0] != '102':
#         return -1
#     return math.floor((int(ip.split(".")[-1])-2) / 5)


# def get_real_label(dic):
#     data = dic.keys()
#     result = np.array([ip_to_group(xi) for xi in data])
#     return result


# answers = get_real_label(flows_ts_ip_total)


def my_dtw(ts1, ts2):
    distance, path = fastdtw(ts1, ts2)
    return distance


def my_pl_ts(ts1, ts2, ip1, ip2):
    return my_dtw(ts1, ts2)


def my_dist(ts1, ts2, ip1="", ip2=""):
    return my_pl_ts(ts1, ts2, ip1, ip2)


def ip_to_user(ip, group_size=5, starting=5):
    isp = int(int(ip.split(".")[-2]))
    node_number = int(ip.split(".")[-1]) - starting - isp
    user = node_number % group_size
    group = math.floor(node_number / group_size)
    return '/tordata/config/group_' + str(group) + "_user_" + str(user)


def compare_ts(ts1, ts2, debug=False):
    # dtw_classic, path_classic = dtw(ts1, ts2, dist='square',
    #                             method='classic', return_path=True)
    # return dtw_classic
    # print(ts1)
    # print(ts2)
    # dist, lag = cross_cor(pd.Series(ts1), pd.Series(ts2))
    dist, lag = cross_cor(ts1, ts2, debug=debug)
    # assert dist >= -1 and dist <= 1
    dist = dist * -1  # flip for use as distance metric
    # assert dist >= -1 and dist <= 1
    return dist, lag


def normalize_ts(ts):
    ts = (ts-ts.min())/(ts.max()-ts.min())
    return ts.fillna(0)


def compare_ts_reshape(ts1, ts2, debug=False):
    # buffer_room = 120  # in seconds
    range = min(ts2.index.values), max(ts2.index.values)
    ts1 = ts1.loc[(ts1.index >= range[0]) & (ts1.index <= range[1])]
    # ts1 = ts1[(ts1['frame.time'] >= int(range[0])) &
    #           (ts1['frame.time'] <= int(range[1]))]
    # print(ts1)
    # ts1 = ts1.loc[:, 'tda_pl']
    ts1 = ts1.values[:, 0]

    ts1_norm = np.array(ts1)
    ts2_norm = np.array(ts2)

    # delay = 0

    # ts1_norm.index = ts1_norm.index + pd.DateOffset(seconds=delay)

    # lock to same range with buffer room
    # on each side to account for network (or PPT) delay

    # detect if no overlap
    if len(ts1_norm) < 2 or len(ts2_norm) < 2:
        return float("inf"), 0

    # Normalize peaks?
    # ts1_norm = normalize_ts(ts1_norm)
    # ts2_norm = normalize_ts(ts2_norm)

    # plot_ts(ts1_norm, ts2_norm)
    # exit(1)

    # else:
    #     ts1_norm = ts1_norm.tolist()
    #     ts2_norm = ts2_norm.tolist()

    score, lag = compare_ts(ts1_norm, ts2_norm, debug=debug)

    return score, lag


def evaluate(src_raw, dst_raw, src_features, dst_feaures, display=False, params=TDA_Parameters(0, 3, 1, 1, 1)):
    src = {}
    dst = {}
    for ip in src_raw:
        try:
            data = src_raw[ip][src_features].copy(deep=True)
        except Exception:
            data = pd.DataFrame(0, index=src_raw[ip].index, columns=src_features)
        if config['tda']:
            src[ip] = ts_to_tda(data)
        else:
            src[ip] = data
    for user in dst_raw:
        try:
            data = dst_raw[user][dst_feaures].copy(deep=True)
        except Exception:
            data = pd.DataFrame(0, index=dst_raw[user].index, columns=dst_feaures)
        if config['tda']:
            dst[user] = ts_to_tda(data)
        else:
            dst[user] = data

    correct = 0.0
    rank_list = []
    score_list = []
    recall_2 = 0
    recall_4 = 0
    recall_8 = 0
    rank = 0
    for user in dst:
        best_score = 0
        best_user = 0
        heap = []
        counter = 0
        r2 = False
        r4 = False
        r8 = False
        for ip in src:
            counter += 1
            score, _ = compare_ts_reshape(src[ip], dst[user])
            if not math.isnan(score) and not math.isinf(score):
                heapq.heappush(heap, (score, counter, ip_to_user(ip)))
            if score < best_score:
                best_score = score
                best_user = ip_to_user(ip)
        if user == best_user:
            correct += 1
        # print(user)
        if recall_at_k(heap.copy(), 2, user):
            recall_2 += 1
            r2 = True
        if recall_at_k(heap.copy(), 4, user):
            recall_4 += 1
            r4 = True
        if recall_at_k(heap.copy(), 8, user):
            recall_8 += 1
            r8 = True
        if (r2 and (not r4 or not r8)) or (r4 and not r8):
            print("r2: " + str(r2))
            print("r4: " + str(r4))
            print("r8: " + str(r8))
            raise Exception("Bad recall")
        rank += get_value_position(heap, user)
        rank_list += [(get_value_position(heap, user), user)]
        score_list += [(heap_to_ordered_list(heap), user)]
    accuracy = correct / len(src)
    recall_2 = recall_2 / len(src)
    recall_4 = recall_4 / len(src)
    recall_8 = recall_8 / len(src)
    rank = rank / len(src)
    return accuracy, recall_2, recall_4, recall_8, rank, rank_list, score_list


def findsubsets(s, n):
    return list(itertools.combinations(s, n))


def evaluate_subset(src_df, dst_df, src_features, dst_feaures, tda_config=None):
    score = evaluate(src_df, dst_df, list(src_features), list(dst_feaures), params=tda_config)
    return score, src_features


def get_features(df):
    features = []
    for src in df:
        features += df[src].columns.tolist()
    return list(set(features))


def iterate_features(src_df, dst_df, n, dst_features, tda_config, filename):
    features = get_features(src_df)
    subsets = findsubsets(features, n)
    results = []
    print("Using " + str(num_cpus) + " cpus for " + str(len(subsets)) + " subsets")
    with mp.Pool(processes=num_cpus) as pool:
        results = []
        for subset in subsets:
            results.append(pool.apply_async(evaluate_subset, args=(src_df, dst_df, subset, dst_features, tda_config)))
        with open(filename, 'a+') as f:
            for result in tqdm(results, total=len(subsets)):
                score, subset = result.get()
                out = str(score) + "\t" + str(subset) + "\n"
                f.write(out)


src_df = flows_ts_ip_total
dst_df = client_chat_logs

# dst_features = ['count']
# src_features = ['count']
# n = 1
# data = evaluate_subset(src_df, dst_df, src_features, dst_features)[-2][-1]
# with open(output_file, 'w') as f:
#     for i in data:
#         out = str(i[-1]) + ", " + str(i[0]) + "\n"
#         f.write(out)

for output_size in range(1, len(dst_df)+1):
    for n in range(1, 3):
        for features in findsubsets(get_features(dst_df), output_size):
            print("Evaluating " + str(n) + " features from " + str(output_size) + " output features")
            best_features = iterate_features(src_df, dst_df, n, features, tda_config,
                                             config['experiment_name'] + str(n) +
                                             "_outputFeatures_" + str(features) +
                                             "_" + str(datetime.now()) +
                                             ".output")
